# Sepsis Challenge 2019 — Feature Engineering & Model

Este cuaderno construye una **Baseline** para el *PhysioNet/Computing in Cardiology Sepsis Challenge 2019* utilizando los archivos Parquet obtenidos:

- `all_patients_setA.parquet`  
- `all_patients_setB.parquet`  

## Pasos realizados
1. **Carga de datos y verificaciones básicas**  
2. **Preprocesamiento de series de tiempo a nivel de paciente**  
3. **Ingeniería de características simple (relleno hacia adelante, medias móviles, deltas)**  
4. **División de entrenamiento/validación por paciente**  
5. **Clasificador base (Regresión Logística / HistGradientBoosting)**  
6. **Evaluación (AUROC, Precisión Promedio)**  
7. **Persistencia del modelo**  

> **Nota:** Este es un punto de partida. Para obtener un rendimiento competitivo necesitarás una definición cuidadosa de las etiquetas alrededor del momento de inicio, validación cruzada por paciente, umbrales calibrados y, potencialmente, modelos secuenciales.

In [1]:

import os
import json
import math
import numpy as np
import pandas as pd
from pathlib import Path

from sklearn.model_selection import GroupShuffleSplit
from sklearn.metrics import roc_auc_score, average_precision_score, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.experimental import enable_hist_gradient_boosting  # noqa: F401
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from joblib import dump


/Users/danielayala/Documents/Daniel Ayala/Study/MaIA/DesarrolloSoluciones/Microproyecto/physionet-sepsis-forecasting/venv/lib/python3.12/site-packages/sklearn/experimental/enable_hist_gradient_boosting.py:16: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


## Archivos .parquet del proyecto Sepsis

In [2]:
DATA_DIR = Path("../data/raw")
FILE_A = DATA_DIR / "all_patients_setA.parquet"
FILE_B = DATA_DIR / "all_patients_setB.parquet"
assert FILE_A.exists(), f"Missing {FILE_A.resolve()}"
assert FILE_B.exists(), f"Missing {FILE_B.resolve()}"

print("Found files:")
print(FILE_A.resolve())
print(FILE_B.resolve())

Found files:
/Users/danielayala/Documents/Daniel Ayala/Study/MaIA/DesarrolloSoluciones/Microproyecto/physionet-sepsis-forecasting/data/raw/all_patients_setA.parquet
/Users/danielayala/Documents/Daniel Ayala/Study/MaIA/DesarrolloSoluciones/Microproyecto/physionet-sepsis-forecasting/data/raw/all_patients_setB.parquet


## Carga de datos

In [3]:
df_a = pd.read_parquet(FILE_A)
df_b = pd.read_parquet(FILE_B)
df = pd.concat([df_a, df_b], ignore_index=True)

print(df.shape)
df.head()

(1552210, 43)


,patient_id,AST,Age,Alkalinephos,BUN,BaseExcess,Bilirubin_direct,Bilirubin_total,Calcium,Chloride,...,SBP,SaO2,SepsisLabel,Temp,TroponinI,Unit1,Unit2,WBC,pH,subHR
0,p000001,NaN,83.14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,p000001,NaN,83.14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,98.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,p000001,NaN,83.14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,122.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,p000001,NaN,83.14,NaN,NaN,24.0,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,7.36,NaN
4,p000001,NaN,83.14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,122.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df.columns

Index(['patient_id', 'AST', 'Age', 'Alkalinephos', 'BUN', 'BaseExcess',
       'Bilirubin_direct', 'Bilirubin_total', 'Calcium', 'Chloride',
       'Creatinine', 'DBP', 'EtCO2', 'FiO2', 'Fibrinogen', 'Gender', 'Glucose',
       'HCO3', 'HR', 'Hct', 'Hgb', 'HospAdmTime', 'ICULOS', 'Lactate', 'MAP',
       'Magnesium', 'O2Sat', 'PTT', 'PaCO2', 'Phosphate', 'Platelets',
       'Potassium', 'Resp', 'SBP', 'SaO2', 'SepsisLabel', 'Temp', 'TroponinI',
       'Unit1', 'Unit2', 'WBC', 'pH', 'subHR'],
      dtype='object')

## Identificacion de las columnas

In [6]:
# Columnas completas en el dataset (43 en total)
all_columns = df.columns

# Asignar directamente las columnas clave
patient_col = "patient_id"
time_col    = "ICULOS"         # número de horas en UCI (es la variable de tiempo en este dataset)
label_col   = "SepsisLabel"    # etiqueta binaria

print("Columnas clave definidas ->")
print("Paciente:", patient_col)
print("Tiempo:", time_col)
print("Etiqueta:", label_col)

# Definir las columnas de características (todas menos ID, tiempo y etiqueta)
non_feature_cols = {patient_col, time_col, label_col}
num_cols = [c for c in all_columns if c not in non_feature_cols]

print(f"Total de features numéricas: {len(num_cols)}")
print(num_cols[:20])  # muestra algunas

Columnas clave definidas ->
Paciente: patient_id
Tiempo: ICULOS
Etiqueta: SepsisLabel
Total de features numéricas: 40
['AST', 'Age', 'Alkalinephos', 'BUN', 'BaseExcess', 'Bilirubin_direct', 'Bilirubin_total', 'Calcium', 'Chloride', 'Creatinine', 'DBP', 'EtCO2', 'FiO2', 'Fibrinogen', 'Gender', 'Glucose', 'HCO3', 'HR', 'Hct', 'Hgb']


## Limpieza de datos y organización de los datos

In [7]:
# Ordernar por paciente y tiempo
df = df.sort_values([patient_col, time_col]).reset_index(drop=True)

# Matener variable númericas para el modelo
non_feature_cols = {patient_col, time_col, label_col}
num_cols = [c for c in df.columns if c not in non_feature_cols and pd.api.types.is_numeric_dtype(df[c])]

print("Número de variables númericas:", len(num_cols))
num_cols[:20]

Número de variables númericas: 40


['AST',
 'Age',
 'Alkalinephos',
 'BUN',
 'BaseExcess',
 'Bilirubin_direct',
 'Bilirubin_total',
 'Calcium',
 'Chloride',
 'Creatinine',
 'DBP',
 'EtCO2',
 'FiO2',
 'Fibrinogen',
 'Gender',
 'Glucose',
 'HCO3',
 'HR',
 'Hct',
 'Hgb']

In [8]:
nan_summary = (
    df.isna()
      .sum()
      .to_frame("NaN_count")
      .assign(NaN_percent=lambda d: d["NaN_count"] / len(df) * 100)
      .sort_values("NaN_percent", ascending=False)
)

# Mostrar las primeras 20 columnas con más NaN
nan_summary.head(20)

,NaN_count,NaN_percent
subHR,1552185,99.998389
Bilirubin_direct,1549220,99.807371
Fibrinogen,1541968,99.340167
TroponinI,1537429,99.047745
Bilirubin_total,1529069,98.509158
Alkalinephos,1527269,98.393194
AST,1527027,98.377604
Lactate,1510764,97.329872
PTT,1506511,97.055875
SaO2,1498649,96.549372


In [9]:
# 1. Eliminar columnas con >90% NaN
threshold = 0.9
cols_to_drop = nan_summary[nan_summary["NaN_percent"] > threshold * 100].index.tolist()
df = df.drop(columns=cols_to_drop)

print("Columnas eliminadas:", cols_to_drop)

Columnas eliminadas: ['subHR', 'Bilirubin_direct', 'Fibrinogen', 'TroponinI', 'Bilirubin_total', 'Alkalinephos', 'AST', 'Lactate', 'PTT', 'SaO2', 'EtCO2', 'Phosphate', 'HCO3', 'Chloride', 'BaseExcess', 'PaCO2', 'Calcium', 'Platelets', 'Creatinine', 'Magnesium', 'WBC', 'BUN', 'pH', 'Hgb', 'FiO2', 'Hct', 'Potassium']


In [10]:
non_feature_cols = {patient_col, time_col, label_col}
num_cols = [c for c in df.columns if c not in non_feature_cols and pd.api.types.is_numeric_dtype(df[c])]

print("Número de variables númericas:", len(num_cols))
num_cols[:20]

Número de variables númericas: 13


['Age',
 'DBP',
 'Gender',
 'Glucose',
 'HR',
 'HospAdmTime',
 'MAP',
 'O2Sat',
 'Resp',
 'SBP',
 'Temp',
 'Unit1',
 'Unit2']

## Generación de nuevas caracteristicas

In [11]:
# Rellenar hacia adelante (forward fill) los valores faltantes dentro de cada paciente
# Esto asegura que si falta un valor en una hora, se tome el último valor conocido
df[num_cols] = df.groupby(patient_col, group_keys=False)[num_cols].apply(lambda g: g.ffill())

# Lista de signos vitales más relevantes para generar nuevas características
#likely_vitals = [
#    "HR","O2Sat","Temp","SBP","MAP","DBP","Resp","EtCO2",
#    "Lactate","Hct","Hgb","Platelets","Glucose"
#]
likely_vitals = [
    "DBP", "Glucose", "HR","SBP","MAP", "O2Sat","Resp", "Temp",
]

# Tomar solo los que realmente existan en el dataset
roll_base = [c for c in likely_vitals if c in df.columns]

# Para cada vital seleccionado, crear una nueva variable con el promedio móvil
# de las últimas 3 horas (ventana=3). Esto captura la tendencia reciente.
for c in roll_base:
    rname = f"{c}_roll3"
    df[rname] = (
        df.groupby(patient_col)[c]
          .transform(lambda s: s.rolling(window=3, min_periods=1).mean())
    )

# Para cada vital seleccionado, crear una nueva variable con el delta (diferencia)
# entre la hora actual y la hora anterior. Esto mide cambios bruscos.
for c in roll_base:
    dname = f"{c}_delta"
    df[dname] = df.groupby(patient_col)[c].diff()

# Actualizar la lista de features numéricas:
# incluir todas las columnas numéricas menos ID, tiempo y etiqueta
num_cols = [
    c for c in df.columns
    if c not in {patient_col, time_col, label_col}
    and pd.api.types.is_numeric_dtype(df[c])
]

# Vista rápida de las primeras filas con las nuevas columnas creadas
df.head()


,patient_id,Age,DBP,Gender,Glucose,HR,HospAdmTime,ICULOS,MAP,O2Sat,...,Resp_roll3,Temp_roll3,DBP_delta,Glucose_delta,HR_delta,SBP_delta,MAP_delta,O2Sat_delta,Resp_delta,Temp_delta
0,p000001,83.14,NaN,0.0,NaN,NaN,-0.03,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,p000001,83.14,NaN,0.0,NaN,97.0,-0.03,2.0,75.33,95.0,...,19.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,p000001,83.14,NaN,0.0,NaN,89.0,-0.03,3.0,86.00,99.0,...,20.500000,NaN,NaN,NaN,-8.0,24.0,10.67,4.0,3.0,NaN
3,p000001,83.14,NaN,0.0,NaN,90.0,-0.03,4.0,86.00,95.0,...,23.666667,NaN,NaN,NaN,1.0,0.0,0.00,-4.0,8.0,NaN
4,p000001,83.14,NaN,0.0,NaN,103.0,-0.03,5.0,91.33,88.5,...,25.500000,NaN,NaN,NaN,13.0,0.0,5.33,-6.5,-5.5,NaN


In [12]:
nan_summary = (
    df.isna()
      .sum()
      .to_frame("NaN_count")
      .assign(NaN_percent=lambda d: d["NaN_count"] / len(df) * 100)
      .sort_values("NaN_percent", ascending=False)
)

# Mostrar las primeras 20 columnas con más NaN
nan_summary.head(20)

,NaN_count,NaN_percent
Unit1,611960,39.425078
Unit2,611960,39.425078
DBP_delta,353521,22.775333
DBP_roll3,320596,20.654164
DBP,320596,20.654164
Glucose_delta,265644,17.113921
Glucose,226888,14.617094
Glucose_roll3,226888,14.617094
Temp_delta,155231,10.000644
Temp,115179,7.420323


## Train/Validation Split by Patient (Group-wise)

In [13]:
# Usa una división por grupos para que ningún paciente aparezca tanto en entrenamiento como en validación.
groups = df[patient_col].values
X = df[num_cols].values
y = df[label_col].values

gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_idx, valid_idx = next(gss.split(X, y, groups=groups))

X_train, X_valid = X[train_idx], X[valid_idx]
y_train, y_valid = y[train_idx], y[valid_idx]

len_train = len(train_idx)
len_valid = len(valid_idx)
len_train, len_valid

(1241213, 310997)

## Primero Modelo (Logistic Regression)

In [14]:
logreg_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler(with_mean=False)),  # sparse safety
    ("clf", LogisticRegression(max_iter=3000, class_weight="balanced", n_jobs=None))
])

logreg_pipe.fit(X_train, y_train)

pred_proba_lr = logreg_pipe.predict_proba(X_valid)[:, 1]
auroc_lr = roc_auc_score(y_valid, pred_proba_lr)
auprc_lr = average_precision_score(y_valid, pred_proba_lr)

print(f"LogReg AUROC: {auroc_lr:.4f} | AUPRC: {auprc_lr:.4f}")

LogReg AUROC: 0.6569 | AUPRC: 0.0370


## Modelo Alternativo (HistGradientBoostingClassifier)

In [ ]:
hgb = HistGradientBoostingClassifier(
    max_depth=None,
    learning_rate=0.05,
    max_iter=300,
    class_weight="balanced"
)
# Impute only (tree models don't need scaling)
imp = SimpleImputer(strategy="median")
X_train_imp = imp.fit_transform(X_train)
X_valid_imp = imp.transform(X_valid)

hgb.fit(X_train_imp, y_train)
pred_proba_hgb = hgb.predict_proba(X_valid_imp)[:, 1]
auroc_hgb = roc_auc_score(y_valid, pred_proba_hgb)
auprc_hgb = average_precision_score(y_valid, pred_proba_hgb)

print(f"HGB AUROC: {auroc_hgb:.4f} | AUPRC: {auprc_hgb:.4f}")

## Threshold y Reporte

In [22]:
def report_at_threshold(y_true, y_prob, thr=0.5, name="model"):
    y_pred = (y_prob >= thr).astype(int)
    print(f"\n{name} @ threshold={thr:.2f}")
    print(classification_report(y_true, y_pred, digits=3))

report_at_threshold(y_valid, pred_proba_lr, thr=0.5, name="LogReg")
#report_at_threshold(y_valid, pred_proba_hgb, thr=0.5, name="HGB")



LogReg @ threshold=0.50
              precision    recall  f1-score   support

         0.0      0.990     0.692     0.814    305750
         1.0      0.033     0.609     0.062      5247

    accuracy                          0.690    310997
   macro avg      0.512     0.650     0.438    310997
weighted avg      0.974     0.690     0.802    310997



## Save Models

In [15]:
Path("models").mkdir(exist_ok=True)
dump(logreg_pipe, "../models/logreg_baseline.joblib")
#dump({"imputer": imp, "model": hgb}, "models/hgb_baseline.joblib")
print("Saved models to ./models/")


Saved models to ./models/
